In [16]:
# 9 - 6 Optional Args in Decorator: 
from functools import wraps, partial
def d(func=None, decorator_arg="dec"):
    if func is None: 
        return partial(d, decorator_arg=decorator_arg)
    
    @wraps(func)
    def wrap(*args, **kwargs):
        print("wrapped", decorator_arg)
        func(*args, **kwargs)
    return wrap

# when there's decorator_arg in decorator, then decoration becomes d(args)(func)
# so we need a decorator without args, so we can do func = d(func). 
# partial does the trick
@d(decorator_arg="another decorator_arg")
def foo(a, b="hello"):
    # You can print the current function's name
    print(foo.__name__)
# foo = d(foo, decorator_arg="another decorator_arg")

foo(2,3)

wrapped another decorator_arg
foo


In [1]:
# 9 - 7 Type checking
# 1 simple structure
from functools import partial, wraps
def type_check(*types):
    def decorate(func):
        @wraps(func)
        def wrap(*args, **kwargs):
            return func(*args, **kwargs)
        return wrap
    return decorate

# So you can add layers on top of the decorator layer, so another way to add optional args
@type_check()
def foo(a, b="hello"):
    print(foo.__name__, "a: ", a)

foo(1)
        
# 2 type checking 
from inspect import signature 
# The mechanism is to bind function with their args signature. 
# So we can compare intended types as a partially bound signature, 
# Actual function as a fully bound signature 
def type_check(func, fully_bound_func, *desired_types):
    sig = signature(func)
    bound_types = sig.bind_partial(*desired_types).arguments
    # sig.bind.arguments = OrderedDict([(arg_name, arg_val) ... ])
    for arg_name, arg_value in fully_bound_func.arguments.items():
        if arg_name in bound_types:
            
            if not isinstance(arg_value, bound_types[arg_name]):
                print(f"Type check error, for {arg_name}, \
                      want: {bound_types[arg_name]}, got {arg_value}")


# bool will be considered an int
type_check(foo, signature(foo).bind(22.4), int)    


foo a:  1
Type check error, for a,                       want: <class 'int'>, got 22.4


In [4]:
# 9-8 Define Decorator as part of a class 
class Foo:
    def dec1(self, func):
        def wrapped(*args, **kwargs):
            print("dec1: ")
            func(*args, **kwargs)
        return wrapped
    
    @staticmethod
    def dec2(func):
        def wrapped(*args, **kwargs):
            print("dec2: ")
            func(*args, **kwargs)
        return wrapped
    
@Foo.dec2
def foo(a, b):
    print("foo")

foo(1,2)

dec2: 
foo


In [11]:
# 9-9.1 Property 
class Foo:
    @property
    def rico(self):
        try:
            return self.val
        except AttributeError:
            return "rico not set"
    @rico.setter
    def rico(self, value):
        self.val = value
    
    # A property decorator is to define a property object like this.
    another_attr = property()
    @another_attr.getter
    def another_attr(self):
        try:
            return self.another_val
        except AttributeError:
            return "another attr not set"
    
    @another_attr.setter
    def another_attr(self, value):
        self.another_val = value

f = Foo()
f.rico = "random val"
f.rico

f.another_attr


'another attr not set'

In [26]:
# 9-9 Decorator implemented as a class (Pre-cursor of descriptor)
# Profiling Layer
from functools import wraps

class Profile:
    def __init__(self, func):
        wraps(func)(self)
        self.ncalls = 0
    def __call__(self, *args, **kwargs):
        self.ncalls = self.ncalls + 1
        return self.__wrapped__(*args, **kwargs)
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            from types import MethodType
            return MethodType(self, instance)
    
@Profile
def some_func(a,b):
    return a,b

some_func(1,2)
some_func(1,2)
some_func(1,2)
# see 3
some_func.ncalls
# Profile object, since it's been wrapped.
type(some_func)

class Spam:
    spam_attr = "hello spam"
    # methods in a class are function objects, and needs to be bound to an instance of Spam. 
    # This is achieved by calling spam.__get__(s, x). A descriptor protocol
    # Since it's been wrapped as Profile object, Profile class needs __get__ as well.
    # MethodType is to bind the function with the corresponding object
    @Profile
    def spam(self, x):
        return x

s = Spam()
s.spam(2)

# To illustrate __get__ : class calls its method's __get__ to bind
def spam_outside_class(self,x):
    print(self.spam_attr)
    return x
# This creates a bound function to Spam, and calls the function.
spam_outside_class.__get__(s, Spam)(2)


hello spam


2

In [29]:
# 9 - 10 static method must wrap the decorated function, not the other way around 
def dec(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
    return wrapper

class Foo:
    @staticmethod
    @dec
    def foo():
        print(foo.__name__)
Foo.foo()

wrapped


In [41]:
# 9 - 11 
# Adding more args while not breaking the wrapped function signature
def dec(func):
    def wrapper(*args, debug = False, **kwargs):
        if debug:
            print("debugging")
        func(*args, **kwargs)
    return wrapper

@dec
def some_func(a,b):
    print(a,b)

some_func(1,2)
# We added another arg here!
some_func(3,4, debug=True)

# 9 - 12 patch a class function with class decorator
def class_dec(cls):
    original_getattribute = cls.__getattribute__
    def new_getattribute(self, name):
        print("new attribute")
        return original_getattribute(self, name)
    cls.__getattribute__ = new_getattribute
    return cls

@class_dec
class Foo:
    bar = 1
    def __init__(self) -> None:
        self.baz = 3
    def spam(self):
        print("spam")

# now can see "new attribute"
Foo().spam()
# this is a class variable, but not going through __getattribute__
# One big reason to use getattr is because we can supply a default value
print(getattr(Foo, "bar", "default string"))
print(getattr(Foo(), "baz", "default string"))

1 2
debugging
3 4
new attribute
spam
new attribute


3

In [1]:
# 9 - 13 metaclass 
# Simple example 1
from typing import Any
# metaclass requires to be child class of type. Otherwise "NoInstances() takes no arguments" in metaclass declaration
class NoInstances(type):
    # So an instance of class NoInstances, the subsequent class, cannot be instantiated as it could be seen as a callable.
    def __call__(self, *args: Any, **kwargs: Any): 
        raise TypeError("No instances pls")

class Spam(metaclass = NoInstances):
    def __init__(self) -> None:
       pass 
    
    @staticmethod
    def foo():
        print("foo")
        
    def __call__(self, *args: Any, **kwargs: Any): 
        print("yeehaw")

Spam.foo()

try:
    s = Spam()
except TypeError as e:
    print(e)

# example 2
class Singleton(type):
    def __init__(cls, *args: Any, **kwargs: Any):
        cls._instance = None

    def __call__(cls, *args: Any, **kwargs: Any): 
        if cls._instance is None:
            print("none")
            # super (class, object)
            # = super(Singleton, cls).__call__(*args, *kwargs)
            # = type(Singleton).__call__(cls,*args, **kwargs), which calls ctor of cls
            cls._instance = super().__call__(*args, **kwargs)
            
        print(f"args: {args}, kwargs: {kwargs}, cls:{cls}")
        return cls._instance

class Foo(metaclass = Singleton):
    pass

f = Foo()
f2 = Foo()
type(f)

foo
No instances pls
none
args: (), kwargs: {}, cls:<class '__main__.Foo'>
args: (), kwargs: {}, cls:<class '__main__.Foo'>


__main__.Foo

In [12]:
# 9-14 seqeuence of calling: __prepare__ -> execution of the class body, to fill the dict with the attrs  
# -> meta meta's __call__ -> __new__ -> __call__ -> __init__
class M(type):
    def __call__(mmcls, *args, **kwargs):
        print("M's call", args, kwargs)
        return super().__call__(*args, **kwargs)

class MM(type, metaclass=M):
    def __prepare__(cls, *args, **kw):
        print("MM Prepare")
        return {}
    def __new__(mcls, *args, **kw):
        print("MM __new__")
        return super().__new__(mcls, *args, **kw)
    def __call__(mmcls, *args, **kwargs):
        print("MM's call", args, kwargs)
        return super().__call__(*args, **kwargs)

class klass(metaclass=MM):
    def __init__(self) -> None:
        print(f"{self.__class__.__name__} initialized")

klass()

MM Prepare
M's call ('klass', (), {'__module__': '__main__', '__qualname__': 'klass', '__init__': <function klass.__init__ at 0x7f12641b8310>}) {}
MM __new__
MM's call () {}
klass initialized


In [35]:
# 9 - 14 Get order of initialization
from collections import OrderedDict
class Meta(type):
    @classmethod
    def __prepare__(cls, clsname, bases):
        # cls is Meta
        print("cls: ", cls, " bases: ", bases)
        return OrderedDict()
    def __new__(cls, clsname, bases, clsdict):
        # converting clsdict back to Dict.
        d = dict(clsdict)
        # adding an item in dict from prepare will actually become a new attribute in the function!
        # methods, class variables will go into clsdict (will be available at new), while regular instance attributes won't 
        d["_order"] = [k for k, _ in clsdict.items()]
        #TODO
        print(f'd: {d}')
        # Equivalent to
        # return super().__new__(cls, clsname, bases, d)
        return type.__new__(cls, clsname, bases, d)
        
class Foo(metaclass = Meta):
    lol_2 = 3
    lol = 0
    def __init__(self):
        self.lol = 1
        print(self.__class__.__name__)
Foo()._order

cls:  <class '__main__.Meta'>  bases:  ()
d: {'__module__': '__main__', '__qualname__': 'Foo', 'lol_2': 3, 'lol': 0, '__init__': <function Foo.__init__ at 0x7f127414c670>, '_order': ['__module__', '__qualname__', 'lol_2', 'lol', '__init__']}
Foo


['__module__', '__qualname__', 'lol_2', 'lol', '__init__']

In [43]:
# 9 - 15: This is to control certain behaviors when a class is created. 
# Requires all 3 functions to have the arg. 
# Another alternative is to pass in a class var in Foo (see 9 - 14), but that'd be available at new and after.
class Meta(type):
    @classmethod
    def __prepare__(cls, clsname, bases, debug = False):
        if debug:
            print("__prepare__")
        return super().__prepare__(clsname, bases)

    def __new__(cls, name, bases, clsdict, debug = False):
        if debug:
            print("__new__")
        return super().__new__(cls, name, bases, clsdict)
    
    def __init__(self, name, bases, ns, debug = False):
        return super().__init__(name, bases, ns)

# Now you now here you add args for metaclass to control class creation behaviors
class Foo(metaclass = Meta, debug = True):
    # another alternative is in __new__, read the class var.
    # debug = True
    def __init__(self):
        print("Foo")

Foo()

__prepare__
__new__
Foo


In [68]:
# 9 - 16, do signature check 
# Simple example
from inspect import Signature, Parameter 
def make_sig(*args):
    params = [Parameter(name,Parameter.POSITIONAL_OR_KEYWORD) for name in args]
    return Signature(params)
def foo(*args, **kwargs):
    signature = make_sig("x", "y", "z")
    # Because we are using bind instead of bind_partial, args, kwargs must be strings EXACTLY the same as the signature.
    bounded_vals=  signature.bind(*args, **kwargs)

# Too many args
# foo(1,2,5,6)
foo(1,2,3)

# More practical example - Structure with common structure of things
class Structure:
    _fields = []
    def __init__(self, *args, **kwargs):
        __sigs__ =  make_sig(*self._fields)
        bounded_vals = __sigs__.bind(*args, **kwargs)
        for name, val in bounded_vals.arguments.items():
            setattr(self, name, val)

class Stock(Structure):
    _fields = ["x", "y"]
    
Stock(x=1,y=2)
    